# Model Pipeline
1. Run clean data script
4. Run Models

In [7]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_validate, train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import Pipeline
from xgboost.sklearn import XGBRegressor
 

In [8]:
# Run clean data script
!python -m data_cleaning.py

/System/Library/Frameworks/Python.framework/Versions/2.7/Resources/Python.app/Contents/MacOS/Python: No module named data_cleaning.py


## View Data
- X: `job, marital, ..., poutcome`
- y: Column `y` with yes or no. 

Can we predict `y` with the features in `X`?

In [9]:
df = pd.read_csv("clean-data.csv")
df.head()

,0,1,2,3,4,5,6,7,8,9,...,1703,1704,1705,1706,1707,1708,1709,1710,1711,1712
0,3.624479,-0.260707,-0.848202,0.090764,-0.957792,-0.275004,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3.624479,-0.559058,-0.906743,0.037863,-0.941831,0.205213,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3.624479,-0.288205,-1.086073,-0.009135,-1.097896,0.156832,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3.624479,-0.616804,-0.906743,0.037863,-0.941831,0.205213,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3.624479,-0.370698,-1.085888,-0.032525,-1.097735,0.132753,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### WTF this? 
Data has been cleaned and preprocessed for a model.
- First column is the target variable.
- Other columns are the features.

### Split data for training

In [ ]:
y = df.iloc[:,0]
X = df.iloc[:,1:]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=0)

# Model Pipeline

###  Want to compare a lot of different models?

In [45]:

models = [LinearRegression(), XGBRegressor(), MLPRegressor(max_iter=100)]
pipeline = Pipeline(steps=[('model', LinearRegression())])
results= []
for classifier in models:
    pipeline.set_params(model = classifier)
    pipeline.fit(X_train, y_train)
    mse = mean_squared_error(y_test, pipeline.predict(X_test))
    print(mse)
    results.append(mse)


1.8321855771650648e+22
44.83490234643944
58.0256951629988


In [54]:
model_names = [str(clf) for clf in models]
pd.DataFrame(results, index = model_names).T

,LinearRegression(),"XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,\n colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,\n importance_type='gain', interaction_constraints='',\n learning_rate=0.300000012, max_delta_step=0, max_depth=6,\n min_child_weight=1, missing=nan, monotone_constraints='()',\n n_estimators=100, n_jobs=4, num_parallel_tree=1, random_state=0,\n reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,\n tree_method='exact', validate_parameters=1, verbosity=None)",MLPRegressor(max_iter=500)
0,1.832186e+22,44.834902,58.025695


WOOH! XGBOOST for the win! Let's just go with this good model.

WTF? Linear Regression does so bad?
- The number of columns in the preprocessed data is 1712 and linear regressions are not that accurate with a large number of parameters.

In [12]:
pd.DataFrame(X_train).shape

(5449, 1712)